In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

[]


In [ ]:
print(os.listdir("../input/train"))
#print(os.listdir("../input/train_images"))

> # Exploring the Data

## Checking the available data

In [ ]:
df_train = pd.read_csv("../input/train/train.csv")
df_train.head()

In [ ]:
df_train.head().loc[2, 'Description']

In [ ]:
df_test = pd.read_csv("../input/test/test.csv")

In [ ]:
print(f"training: {len(df_train)} \t testing: {len(df_test)}")

In [ ]:
import json

In [ ]:
with open("../input/train_sentiment/df880995b.json") as f:
    json_sample = json.load(f)

In [ ]:
json_sample

In [ ]:
df_train.describe(include='all')

In [ ]:
df_train.info()

In [ ]:
from pylab import imread,subplot,imshow,show
import matplotlib.pyplot as plt
image = imread('../input/train_images/86e1089a3-1.jpg') 
plt.imshow(image)


In [ ]:
pd.read_csv('../input/color_labels.csv')

### Notes
* **Breeds, Colors, State** need changing to categories
* **PhotoAmt** could do with changing to an integer
* Missing **Description** need replacing with "none" or "N/A", if used in model
* Explore descriptions in greater depth
* Missing **Name** need replacing with "none" or "N/A", if used in model
* Need to ensure model is suitable for categorical output: classifiers, logistic regression etc
* Categories for **Fur Length**, **Vaccinated** **etc**
* One-Hot Encoding for categories
* Looking at top rows, RescuerID can be the same many times - I suspect this is a pet shelter


## Analysing Description

In [ ]:
df_sample = df_train.head(20)

In [ ]:
df_sample

In [ ]:
from collections import Counter, defaultdict
import string
from nltk.corpus import stopwords

In [ ]:
remove_punc = str.maketrans({key: None for key in string.punctuation})
stopwords = set(stopwords.words('english'))

In [ ]:
def bag_of_words(df, include=None, exclude=[]):
    dct = defaultdict(Counter)
    for i, speed, desc in df[['AdoptionSpeed','Description']].itertuples():
        try:
            desc = desc.translate(remove_punc).lower()
        except AttributeError:
            continue
        for word in desc.split():
            if (include == None or word in include) and word not in exclude:
                dct[speed][word] += 1
    return dct
    

In [ ]:
BoW = bag_of_words(df_train, exclude=stopwords)

In [ ]:
BoW[0].most_common(50)

In [ ]:
all_words = set()
exclude = set()
for speed in range(5):
    freq = BoW[speed].most_common()
    for word,_ in freq:
        if word in all_words:
            exclude.add(word)
        all_words.add(word)
include = all_words - exclude
        

In [ ]:
include

In [ ]:
df_train_speed0 = df_train[df_train['AdoptionSpeed'] == 0]

In [ ]:
BoW = bag_of_words(df_train)

In [ ]:
BoW[4].most_common()

In [ ]:
model_words = set()
for k,v in BoW.items():
    freq = v.most_common(100)
    for word,_ in freq:
        model_words.add(word)
len(model_words)
    

In [ ]:
model_words

In [ ]:
df_train_desc = df_train[['PetID','Description', 'AdoptionSpeed']].copy()
#df_test_desc = df_test[['PetID', 'Description']].copy()

In [ ]:
def apply_word_flags(df, words):
    for word in words:
        df[word] = 0
    for i, desc in df['Description'].items():
        try:
            for word in desc.split():
                if word in words:
                    df.at[i,word] = 1
        except AttributeError:
            continue
    return df

In [ ]:
df_train_words = apply_word_flags(df_train_desc, model_words)
#df_test_words = apply_word_flags(df_test_desc, model_words)

In [ ]:
df_train_words.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train_all = df_train_words.drop(columns=['PetID','AdoptionSpeed','Description'])
y_train_all = df_train_words['AdoptionSpeed']
# X_test = df_test_words.drop(columns=['PetID', 'Description'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, test_size=0.20).copy()

In [ ]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

In [ ]:
prediction = rfc.predict(X_test)

In [ ]:
import ml_metrics

In [ ]:
ml_metrics.quadratic_weighted_kappa(rater_a=prediction, rater_b=y_test)

In [ ]:
feature_importance = rfc.feature_importances_
word_importance = []
for i,word in enumerate(feature_importance):
    word_importance.append((word, X_train.columns[i]))
    
    

In [ ]:
model_words = set()
for score, word in sorted(word_importance, reverse=True):
    if score >= 0.005:
        model_words.add(word)
    else:
        break
model_words

In [ ]:
cats = df_train_desc[df_train_desc['cat'] == 1]
dogs = df_train_desc[df_train_desc['dog'] == 1]

In [ ]:
bins = [0,1,2,3,4, 5]

In [ ]:
plt.hist(cats['AdoptionSpeed'], bins, alpha=0.5, label="cats")
plt.hist(dogs['AdoptionSpeed'], bins, alpha=0.5, label="dogs")
plt.legend(loc='upper right')
plt.show()